In [ ]:
import math
import os
import random
from datetime import date, datetime, timedelta

import faker
import pandas as pd
from datagen_customer import generate_customers
from datagen_transaction import generate_transactions
from faker import Faker

In [ ]:
start_dt = datetime.strptime("2021-03-01", "%Y-%m-%d")
end_dt = datetime.strptime("2021-06-30", "%Y-%m-%d")

initial_cust_base_size = 1000
seed_num = 4444

In [ ]:
fake = Faker()
Faker.seed(seed_num)

We initial with 1000 customers with transactions for 3 months.

In [ ]:
cust_list = pd.DataFrame()
txn_df = pd.DataFrame()

In [ ]:
profiles = [
    p
    for p in os.listdir("profiles")
    if p.startswith("adults") or (p.startswith("young"))
]
profiles

In [ ]:
customer_output_file = "customers_list.csv"
txn_output_file = "_txn.csv"

for dt in (start_dt + timedelta(n) for n in range(int((end_dt - start_dt).days))):
    print("Date", dt)

    if dt == start_dt:
        num_new_customers = initial_cust_base_size
    else:
        increment = random.uniform(0.01, 0.03)
        num_new_customers = math.ceil(len(cust_list) * increment)

    print("Number of customers newly onboard: ", num_new_customers)

    new_cust_list = generate_customers(
        fake, num_new_customers, "profiles/main_config.json"
    )
    cust_list = cust_list.append(new_cust_list, ignore_index=True)
    print("Total customer: ", len(cust_list))
    new_cust_list.to_csv(
        customer_output_file, mode="a", header=not os.path.exists(customer_output_file)
    )

    for p in profiles:
        # generate txns for the new customers from current date to the end date
        new_cust_txn_df = generate_transactions(new_cust_list, p, dt, end_dt)
        print(f"Number of transaction ({p}): ", len(new_cust_txn_df))
        new_cust_txn_df.to_csv(
            f"{p}{txn_output_file}",
            mode="a",
            header=not os.path.exists(f"{p}{txn_output_file}"),
        )

        txn_df = txn_df.append(new_cust_txn_df, ignore_index=True)

In [ ]:
print(f"Number of customers: {len(cust_list)}, Number of transactions: {len(txn_df)}")